In [1]:
# Basic
from collections import Counter
import datetime
import math
import numpy as np
import os
import pprint

# yaml specific
import yaml

# Data handling
from fuzzywuzzy import fuzz, process
import pandas as pd
from tqdm import tqdm

# my library
from db_utils import update_player, add_player 

In [2]:
# Config variables
raw_data_path = "raw_data"
clean_data_path = "clean_data"
tournament_name = "IPL"

### Utility functions. 

In [3]:
def find_players_not_mapped(matches):
    """
    Utility function to quickly run through all deliveries of every match and see if there is any exception coming (if there is any player unmapped)
    Args:
        matches - a list of match objects parsed from the yaml files
    """
    players_not_found_or_mapped = []
    for match in tqdm(matches):
        try:
            for inning in match['innings']:
                for inning_number in inning:
                    for ball in inning[inning_number]['deliveries']:
                        for ball_number in ball:

                            batsman = player_id_map[ball[ball_number]['batsman']]
                            bowler = player_id_map[ball[ball_number]['bowler']]
                            non_striker = player_id_map[ball[ball_number]['non_striker']]

                            if "wicket" in ball[ball_number]:
                                player_dismissed = player_id_map[ball[ball_number]["wicket"]["player_out"]]
                                dismissal_type = ball[ball_number]["wicket"]["kind"]

                                if "fielders" in ball[ball_number]["wicket"]:
                                    # There is "(sub)" when a substitute fielder is involved in a wicket
                                    fielders = ",".join([str(player_id_map[fielder.replace(" (sub)", "")]) for fielder in ball[ball_number]["wicket"]["fielders"]])
        except Exception as e:
            print(f"Exception {e} happened in ball number {ball_number} ")
            print(f"ball {ball[ball_number]}")
            print(f"match info: {match['info']}")
            players_not_found_or_mapped.append(e.args[0])
            
    return players_not_found_or_mapped

In [4]:
def parse_yaml(path):
    """
    Parses a given yaml file and returns the object
    Args:
        path - path of the yaml file to be parsed
    """
    with open(path, 'r') as stream:
        try:
            data = yaml.safe_load(stream)
        except yaml.YAMLError as exc:
            print(exc)
    return data

In [5]:
def get_all_matches_raw_data(tournament_name):
    """
        Parses all the matches inside the tournament_name folder and returns a list of dicts
        Args:
            tournament_name - Name of tournament to be parsed
    """
    
    tournament_path = os.path.join(raw_data_path, tournament_name)
    match_files = os.listdir(tournament_path)
    matches = []
    print(f"Parsing {len(match_files)} matches for {tournament_name}")
    for match_file in tqdm(match_files):
        if ".yaml" not in match_file:
            continue
        match_path = os.path.join(tournament_path, match_file)
        match = parse_yaml(match_path)
        matches.append(match)
    return matches

In [6]:
def get_best_possible_name_match(query_name, players_with_same_surname):
    """
    This function does a fuzzy match to find where to insert query_name in the players database. It returns the best matched player_name
    Args:
        query_name - query name to be checked with
        players_with_same_surname - a subset of the players dataframe for checking with the query_name
    """
    
    #print(f"Getting best match for {query_name}")
    
    # collecting all capital letters in the name to be stored as initials
    initials = [c for c in query_name if c.isupper()]
    
    # creating a hash map between the player_name and player_full_name for easy access
    name_full_name_map = {}
    for index, row in players_with_same_surname.iterrows():
        name_full_name_map[row["player_name"]] = row["player_full_name"]
    
    # List of all choices for the fuzzy algorithm to run on. 
    choices_name = np.array(players_with_same_surname.player_name)
    potential_matches = process.extract(query_name, choices_name, processor=None, limit=50)
    
    # IF no potential match was found, return null string
    if len(potential_matches) == 0:
        return ""
    
    #print(f"All matches: {potential_matches}")
    top_score = potential_matches[0][1]
    best_match = ""
    for potential_match in potential_matches:
        score = potential_match[1]
        match_name = potential_match[0]
    
        # if you get a perfect score, that should definitely be the correct match
        if score == 100:
            best_match = match_name
            break
        
        # if you dont get a perfect score, check if all the initials are present in the players full name, that should be the best match
        initial_bool = []
        for initial in initials:
            #print(f"Checking if {initial} is present in {name_full_name_map[match_name]}")
            if initial in name_full_name_map[match_name]:
                initial_bool.append(True)
            else:
                initial_bool.append(False)
            name_full_name_map[match_name] = name_full_name_map[match_name].replace(initial, '', 1)

        if all(initial_bool):
            best_match = match_name
            break
        
        # if either of methods fail, return the best match the fuzzy scoring algo returned
        if score > top_score:
            top_score = score
            best_match = match_name

    return best_match

# This notebook will be used to curate 3 tables. 
## 1) Venue 2) Match 3) Ball

In [7]:
matches = get_all_matches_raw_data(tournament_name)

  0%|                                                                                                                                                                                                               | 0/818 [00:00<?, ?it/s]

Parsing 818 matches for IPL


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 818/818 [03:47<00:00,  3.60it/s]


#### We need to first match all player names found in this dataset to the player_ids we have. Our player table contains full names (eg: David Warner) while this dataset has only the short names typically used in score cards (eg. DA Warner). So using a combination of fuzzy and boolean matching to map these

In [8]:
players = pd.read_csv(os.path.join(clean_data_path, "player.csv"))

### Creating a unique player names set with information from each ball of all the 817 IPL matches. Considering batsman bowler and non-striker

In [9]:
unique_player_names = set()
for match in tqdm(matches):
    for inning in match['innings']:
        for inning_number in inning:
            for ball in inning[inning_number]['deliveries']:
                for ball_number in ball:
                    unique_player_names.add(ball[ball_number]['batsman'])
                    unique_player_names.add(ball[ball_number]['bowler'])
                    unique_player_names.add(ball[ball_number]['non_striker'])

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 816/816 [00:00<00:00, 3685.57it/s]


In [10]:
len(unique_player_names)

580

### for each one of these unique players, we are trying to find a best match map from the players sheet and updating the player_display_name

In [11]:
for player_name in tqdm(unique_player_names):
    players_with_same_surname = players[players.player_name.str.contains(player_name.split(" ")[-1])]
    best_match_name = get_best_possible_name_match(player_name, players_with_same_surname)
    # if null string was returned, skip the player
    if best_match_name:
        update_player("player_display_name", player_name, "player_name", best_match_name)

 44%|██████████████████████████████████████████████████████████████████████████████████████▌                                                                                                              | 255/580 [00:14<00:18, 18.01it/s]C:\Users\aniru\miniconda3\envs\torch\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 580/580 [00:32<00:00, 17.62it/s]


### Beyond this if we find any inaccuracies in the mapping, lets just manually correct them

## Venue table

In [12]:
unique_venues = set()
all_venues = []
for match in tqdm(matches):
    unique_venues.add(match['info']['venue'])
    all_venues.append(match['info']['venue'])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 816/816 [00:00<00:00, 819576.64it/s]


### Trying to find if there are duplicate names for the same stadium

In [13]:
unique_venues_list = list(unique_venues)
similarity_scores = []
for i in range(len(unique_venues_list)-1):
    for j in range(i+1, len(unique_venues_list)):
        similarity_score = fuzz.WRatio(unique_venues_list[i], unique_venues_list[j])
        similarity_scores.append([i, j, similarity_score])

similarity_scores_sorted = sorted(similarity_scores, key=lambda x : x[2], reverse=True)
counter = 0
for pair in similarity_scores_sorted:
    #print(f"{unique_venues_list[pair[0]]} - {unique_venues_list[pair[1]]} --> {pair[2]}")
    if counter > 20:
        break

### These stadiums have duplicates so removing them, but they need to be mapped to the same venue_id when iterating over the matches

In [14]:
unique_venues_list.remove("M.Chinnaswamy Stadium")
unique_venues_list.remove("Punjab Cricket Association IS Bindra Stadium, Mohali")

In [15]:
venue_columns = ["venue_id", "venue_name"]
venue_csv = pd.DataFrame(columns = venue_columns)
for i in range(len(unique_venues_list)):
    venue_id = i+1
    venue_name = unique_venues_list[i]
    venue_csv = venue_csv.append({"venue_id" : venue_id, 
                                  "venue_name" : venue_name}, ignore_index=True)
venue_csv.to_csv(os.path.join(clean_data_path, "venue.csv"), index=False)

### Utility maps for tournaments, venues, teams and players

In [17]:
df_tournament = pd.read_csv(os.path.join(clean_data_path, "tournament.csv"))
df_tournament = df_tournament.loc[:, ~df_tournament.columns.str.contains('^Unnamed')]
tournament_id_map = dict(zip(df_tournament.tournament_name, df_tournament.tournament_id))

In [18]:
df_venue = pd.read_csv(os.path.join(clean_data_path, "venue.csv"))
df_venue = df_venue.loc[:, ~df_venue.columns.str.contains('^Unnamed')]
venue_id_map = dict(zip(df_venue.venue_name, df_venue.venue_id))

# Mapping the duplicates as well to its correct venue ids
venue_id_map["Punjab Cricket Association IS Bindra Stadium, Mohali"] = venue_id_map["Punjab Cricket Association Stadium, Mohali"]
venue_id_map["M.Chinnaswamy Stadium"] = venue_id_map["M Chinnaswamy Stadium"]

In [20]:
df_team = pd.read_csv(os.path.join(clean_data_path, "team.csv"))
df_team = df_team.loc[:, ~df_team.columns.str.contains('^Unnamed')]
team_id_map = dict(zip(df_team.team_name, df_team.team_id))

### Use this cell for all manual corrections to update/insert in players table

In [21]:
# updates
update_player("player_display_name", "RG Sharma", "player_name", "Rohit Sharma")
update_player("player_display_name", "SA Yadav", "player_name", "Suryakumar Yadav")
update_player("player_display_name", "CRD Fernando", "player_name", "Dilhara Fernando")
update_player("player_display_name", "DPMD Jayawardene", "player_name", "Mahela Jayawardene")
update_player("player_display_name", "R Powell", "player_name", "Rovman Powell")
update_player("player_display_name", "RK Singh", "player_name", "Rinku Singh")
update_player("player_display_name", "JPR Scantlebury-Searles", "player_name", "Javon Searles")
update_player("player_display_name", "Milind Kumar", "player_name", "Milind Kumar")
update_player("player_display_name", "NB Singh", "player_name", "Nathu Singh")
update_player("player_display_name", "AS Yadav", "player_name", "Arjun Yadav")
update_player("player_display_name", "VRV Singh", "player_full_name", "Vikram Raj Vir Singh")
update_player("player_display_name", "R Bishnoi", "player_full_name", "Rajesh Bishnoi")
update_player("player_display_name", "KH Devdhar", "player_full_name", "Kedar Hemant Devdhar")
update_player("player_display_name", "Harmeet Singh (2)", "player_full_name", "Harmeet Singh")
update_player("player_display_name", "AV Wankhade", "player_full_name", "Apoorv Vijay Wankhade")
update_player("player_display_name", "B Aparajith", "player_full_name", "Baba Aparajith")
update_player("player_display_name", "Anmolpreet Singh", "player_full_name", "Anmolpreet Singh")





# inserts
add_player("Ankit Soni", "Ankit Soni", "Ankit Soni", "Right-hand bat", "Legbreak googly", "02/08/1993", "2", str(team_id_map["Gujarat Lions"]))
add_player("J Suchith", "Jagadeesha Suchith", "Jagadeesha Suchith", "Left-hand bat", "Slow left-arm orthodox", "01/16/1994", "2", str(team_id_map["Kings XI Punjab"]))
add_player("SD Lad", "Siddhesh Lad", "Siddhesh Dinesh Lad", "Right-hand bat", "Right-arm offbreak", "23/05/92", "2", str(team_id_map["Mumbai Indians"]))
add_player("S Kaushik", "Shivil Kaushik", "Shivil Sharma Kaushik", "Left-hand bat", "Slow left-arm wrist-spin", "07/09/95", "2", str(team_id_map["Gujarat Lions"]))
add_player("KM Asif", "KM Asif", "KM Asif", "Right-hand bat", "Right-arm medium", "24/07/93", "2", str(team_id_map["Chennai Super Kings"]))
add_player("AS Roy", "Anukul Roy", "Anukul Sudhakar Roy", "Left-hand bat", "Slow left-arm orthodox", "30/11/98", "2", str(team_id_map["Mumbai Indians"]))
add_player("YBK Jaiswal", "Yashasvi Jaiswal", "Yashasvi Bhupendra Kumar Jaiswal", "Left-hand bat", "", "28/12/01", "2", str(team_id_map["Rajasthan Royals"]))
add_player("Abdul Samad", "Abdul Samad", "Abdul Samad", "Right-hand bat", "Right-arm legbreak", "10/28/01", "2", str(team_id_map["Sunrisers Hyderabad"]))
add_player("Kartik Tyagi", "Kartik Tyagi", "Kartik Tyagi", "Right-hand bat", "Right-arm fast", "11/08/00", "2", str(team_id_map["Rajasthan Royals"]))
add_player("Lalit Yadav", "Lalit Yadav", "Lalit Yadav", "Right-hand bat", "Right-arm offbreak", "03/01/97", "2", str(team_id_map["Delhi Daredevils"]))
add_player("Ravi Bishnoi", "Ravi Bishnoi", "Ravi Bishnoi", "Right-hand bat", "Legbreak googly", "05/09/00", "2", str(team_id_map["Kings XI Punjab"]))
add_player("RA Shaikh", "Rahil Shaikh", "Rahil Akhil Ahmed Shaikh", "Left-hand bat", "Left-arm medium", "12/06/85", "2", str(team_id_map["Mumbai Indians"]))
add_player("AN Ahmed", "AN Ahmed", "AN Ahmed", "Left-hand bat", "Left-arm medium", "12/06/85", "2", str(team_id_map["Mumbai Indians"]))
add_player("AA Kazi", "Abrar Kazi", "Abrar Anjum Kazi", "Left-hand bat", "Slow left-arm orthodox", "10/29/89", "2", str(team_id_map["Royal Challengers Bangalore"]))
add_player("T Mishra", "Tanmay Mishra", "Tanmay Mishra", "Right-hand bat", "Right-arm medium-fast", "12/22/86", "2", str(team_id_map["Deccan Chargers"]))




In [22]:
df_player = pd.read_csv(os.path.join(clean_data_path, "player.csv"))
df_player = df_player.loc[:, ~df_player.columns.str.contains('^Unnamed')]
player_id_map = dict(zip(df_player.player_display_name, df_player.player_id))

## Match and Ball table

In [23]:
match_columns = ["match_id", "tournament_id", "venue_id", "match_date", "team_1", "team_2", "toss_winner", "toss_decision", "player_of_match", 
                 "match_winner", "match_win_by_runs", "match_win_by_wickets", "highlights_url", "match_url", "match_description"]
#match_csv = pd.DataFrame(columns = match_columns)

ball_columns = ["ball_id", "match_id", "ball_number", "innings_number", "batsman", "bowler", "non_striker", "batsman_runs", "wide_runs", "noball_runs",
                "bye_runs", "legbye_runs", "extras_runs", "total_runs", "player_dismissed", "dismissal_type", "fielders", "ball_description", "ball_url"]
#ball_csv = pd.DataFrame(columns = ball_columns)

match_csv = {}
ball_csv = {}

i = 0 # match id
j = 0 # ball id
for match in tqdm((matches), position=0, leave=True):
    
    # First adding entries to the match table
    try:
        
        match_id = i
        tournament_id = tournament_id_map[match['info']['competition']] # For ODI, TEST "competition" to be changed to "match_Type"
        venue_id = venue_id_map[match['info']['venue']]
        
        # if date is a date instance, parse and read. if not directly read
        match_date = [",".join([str(date.strftime('%Y-%m-%d')) if isinstance(date, datetime.date) else str(date) for date in match['info']['dates']])]

        team_1_best_fuzzy_match = process.extractOne(match['info']['teams'][0], team_id_map.keys())[0]
        team_2_best_fuzzy_match = process.extractOne(match['info']['teams'][1], team_id_map.keys())[0]
        team_1 = team_id_map[team_1_best_fuzzy_match]
        team_2 = team_id_map[team_2_best_fuzzy_match]

        toss_winner_best_fuzzy_match = process.extractOne(match['info']['toss']['winner'], team_id_map.keys())[0]
        toss_winner = team_id_map[toss_winner_best_fuzzy_match]
        toss_decision = match['info']['toss']['decision']
        
        # Match was canceled/washed out
        if "result" in match['info']['outcome'] and match['info']['outcome']['result'] == "no result":
            match_winner = "NA"
            match_win_by_wickets = "NA"
            match_win_by_runs = "NA"
            player_of_match = "NA"
            
        # Match was tied
        elif "result" in match['info']['outcome'] and match['info']['outcome']['result'] == "tie":
            match_winner = "TIE"
            match_win_by_wickets = "NA"
            match_win_by_runs = "NA"
            player_of_match = ",".join([str(player_id_map[player]) for player in match['info']['player_of_match']])

        elif "runs" in match['info']['outcome']['by']:
            match_win_by_runs = match['info']['outcome']['by']['runs']
            match_win_by_wickets = 'NA'
            match_winner_best_fuzzy_match = process.extractOne(match['info']['outcome']['winner'], team_id_map.keys())[0]
            match_winner = team_id_map[match_winner_best_fuzzy_match]
            player_of_match = ",".join([str(player_id_map[player]) for player in match['info']['player_of_match']])

        elif "wickets" in match['info']['outcome']['by']:
            match_win_by_wickets = match['info']['outcome']['by']['wickets']
            match_win_by_runs = 'NA'
            match_winner_best_fuzzy_match = process.extractOne(match['info']['outcome']['winner'], team_id_map.keys())[0]
            match_winner = team_id_map[match_winner_best_fuzzy_match]
            player_of_match = ",".join([str(player_id_map[player]) for player in match['info']['player_of_match']])
        
        # For ambitious future deep learning projects
        highlights_url = 'NA'
        match_url = 'NA'
        match_description = 'NA'
        
        match_csv[i] = {
                        "match_id" : match_id, 
                        "tournament_id" : tournament_id,
                        "venue_id" : venue_id,
                        "match_date" : match_date, 
                        "team_1" : team_1,
                        "team_2" : team_2,
                        "toss_winner" : toss_winner,
                        "toss_decision" : toss_decision,
                        "player_of_match" : player_of_match,
                        "match_winner" : match_winner,
                        "match_win_by_runs" : match_win_by_runs,
                        "match_win_by_wickets" : match_win_by_wickets,
                        "highlights_url" : highlights_url,
                        "match_url" : match_url,
                        "match_description" : match_description
        }
    
    except Exception as e:
        
        print("Exception happened in match: ", e)
        print(match['info'])
        
        break
    
    
    # Then adding entries to the ball table
    
    try:
        for inning in match['innings']:
            for inning_number in inning:
                for ball in inning[inning_number]['deliveries']:
                    for ball_number in ball:
                        ball_id = j
                        innings_number = inning_number[0]
                        batsman = player_id_map[ball[ball_number]['batsman']]
                        bowler = player_id_map[ball[ball_number]['bowler']]
                        non_striker = player_id_map[ball[ball_number]['non_striker']]
                        
                        wide_runs = 0
                        bye_runs = 0
                        noball_runs = 0
                        legbye_runs = 0
                        batsman_runs = 0
                        extras_runs = 0
                        total_runs = 0
                        
                        player_dismissed = "NA"
                        dismissal_type = "NA"
                        fielders = "NA"
                        
                        if "extras" in ball[ball_number]:
                            if "wides" in ball[ball_number]["extras"]:
                                wide_runs = ball[ball_number]["extras"]["wides"]
                            if "legbyes" in ball[ball_number]["extras"]:
                                legbye_runs = ball[ball_number]["extras"]["legbyes"]
                            if "noballs" in ball[ball_number]["extras"]:
                                noball_runs = ball[ball_number]["extras"]["noballs"]
                            if "byes" in ball[ball_number]["extras"]:
                                bye_runs = ball[ball_number]["extras"]["byes"]
                        
                        if "runs" in ball[ball_number]:
                            batsman_runs = ball[ball_number]["runs"]["batsman"]
                            extras_runs = ball[ball_number]["runs"]["extras"]
                            total_runs = ball[ball_number]["runs"]["total"]
                            
                        if "wicket" in ball[ball_number]:
                            player_dismissed = player_id_map[ball[ball_number]["wicket"]["player_out"]]
                            dismissal_type = ball[ball_number]["wicket"]["kind"]
                            
                            if "fielders" in ball[ball_number]["wicket"]:
                                # There is "(sub)" when a substitute fielder is involved in a wicket
                                fielders = ",".join([str(player_id_map[fielder.replace(" (sub)", "")]) for fielder in ball[ball_number]["wicket"]["fielders"]])
                        
                        # For ambitious future deep learning projects
                        ball_description = "NA"
                        ball_url = "NA"
                        
                        ball_csv[j] = {
                                        "ball_id" : j, 
                                        "match_id" : match_id,
                                        "ball_number" : ball_number,
                                        "innings_number" : innings_number, 
                                        "batsman" : batsman,
                                        "bowler" : bowler,
                                        "non_striker" : non_striker,
                                        "batsman_runs" : batsman_runs,
                                        "wide_runs" : wide_runs,
                                        "bye_runs" : bye_runs,
                                        "noball_runs" : noball_runs,
                                        "legbye_runs" : legbye_runs,
                                        "extras_runs" : extras_runs,
                                        "total_runs" : total_runs,
                                        "player_dismissed" : player_dismissed,
                                        "dismissal_type" : dismissal_type,
                                        "fielders" : fielders,
                                        "ball_description" : ball_description,
                                        "ball_url" : ball_url,
                        }
                        
                        j += 1
    
    except Exception as e:
        print(f"Exception {e} happened in ball number {ball_number} ")
        print(f"ball {ball[ball_number]}")
        print(f"match info: {match['info']}")
        break
        
    i += 1

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 816/816 [01:19<00:00, 10.31it/s]


In [24]:
df_match_csv = pd.DataFrame.from_dict(match_csv, "index")
df_ball_csv = pd.DataFrame.from_dict(ball_csv, "index")

df_match_csv.to_csv(os.path.join(clean_data_path, "match.csv"), index=False)
df_ball_csv.to_csv(os.path.join(clean_data_path, "ball.csv"), index=False)